# Creating a P2WPKH transaction

In this section we'll create a P2WPKH transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and we'll test it using bitcoin core in regtest mode.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).


- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
    - Base58 addresses - '[Addresses chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
    - TestShell setup - '[P2PKH chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter1-legacy/p2pkh.ipynb)'

## Setup 

### Requirements
For this exercise we'll need Bitcoin Core. This notebook has been tested with [v24.0.1](https://github.com/bitcoin/bitcoin/releases/tag/v24.0.1).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing bitcoin-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/Users/dariuscognac/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/Users/dariuscognac/Documents/Github/bitcoin-tx-tutorial"

import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *

import json

### Create a P2WPKH UTXO

In order to create a transaction spending from a P2WPKH UTXO, we'll first need to create the UTXO that is locked with a p2wpkh script. To do that, we'll define a private key, use it to generate a pubkey, then convert the pubkey to a p2wpkh address.

#### Create a p2wpkh address 
For more on this step, review the 'Addresses' notebook.

In [2]:
sender_privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
sender_pubkey = privkey_to_pubkey(sender_privkey)
address_to_spend = pk_to_p2wpkh(sender_pubkey, network = "regtest")
print("sender's p2wpkh address: " + address_to_spend)

sender's p2wpkh address: bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw


#### Fund the 'sender' with 2.001 btc (0.001 btc is for the next tx fee)

Now that we have the address, we'll fund it using the python code we used in the previous sections, `create_regtest_utxo.py`.

In [3]:
node = setup_testshell()
txid_to_spend, index_to_spend = fund_address(node, address_to_spend, 2.001)
print(f"txid: {txid_to_spend}, {index_to_spend}")

2022-12-08T10:18:24.398000Z TestFramework (INFO): Initializing test directory /var/folders/r5/yk8yg2xs1gs8xzkn5l8vr72w0000gn/T/bitcoin_func_test_5i0cu3j8
txid: d4ce50311efa12a97b6c910afba180686687edaec7de92ffcfba18777673991c, 0


## Spending a p2wpkh UTXO

Now that we have some funds locked up in a p2wpkh utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw`.

### Decoding a bech32 address

The first thing we need to do is decode the address by doing this we can achieve the following:
1 - validate the checksum to know the address was transmitted without error
2 - make sure we are sending btc on the correct network (testnet/mainnet)
3 - know what to put in the scriptPubkey

For more on addresses, refer back to the 'Addresses' chapter.

In [4]:
receiver_address = 'bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw'

# human readable part
hrp = 'bcrt'
witver, witprog = b32.decode(hrp, receiver_address)

pubkey_hash = bytearray(witprog)
print("witness version: ", witver)
print("pubkey/script hash: ", pubkey_hash.hex())

witness version:  0
pubkey/script hash:  fc7250a211deddc70ee5a2738de5f07817351cef


The witness version `0`, tells us that this address corresponds to either a p2wpkh or p2wsh output. Then, we can tell from the length of the hash whether it is a pubkey hash (20 bytes long), or a script hash (32 bytes long). 

For more on decoding addresses, refer back to the 'Addresses' chapter.

Now we can create the receiver's output scriptPubkey:

In [5]:
receiver_spk = bytes.fromhex("0014") + pubkey_hash

### Create an unsigned p2wpkh transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [6]:
# Note we have already defined a few variables we need to create our transaction:
# The input utxo txid and index: `txid_to_spend` and `index_to_spend`
# The input private key and public key: `sender_privkey` and `sender_pubkey`

# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("0014") + hash160(change_pubkey)

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the witness field is empty. This first step is necessary as the signature will cover the whole transaction (using SIGHASH_ALL). In a later chapter we will cover other sighash types and how they are signed.

In [7]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  02000000011c9973767718bacfff92dec7aeed87666880a1fb0a916c7ba912fa1e3150ced40000000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d00000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

In [8]:
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "72ed59fd70134b2d5bb45eb16cb77d418ff2407998a30e72796aa775e437b05e",
  "hash": "72ed59fd70134b2d5bb45eb16cb77d418ff2407998a30e72796aa775e437b05e",
  "version": 2,
  "size": 113,
  "vsize": 113,
  "weight": 452,
  "locktime": 0,
  "vin": [
    {
      "txid": "d4ce50311efa12a97b6c910afba180686687edaec7de92ffcfba18777673991c",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "1.50000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "0 fc7250a211deddc70ee5a2738de5f07817351cef",
        "desc": "addr(bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw)#luvpmd0q",
        "hex": "0014fc7250a211deddc70ee5a2738de5f07817351cef",
        "address": "bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw",
        "type": "witness_v0_keyhash"
      }
    },
    {
      "value": "0.50000000",
      "n": 1,
      "scriptPubKey": {
        "asm": "0 531260aa2a199e228c537dfa42c82bea2c7c1f4d

Segwit transactions have a new signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)


In [9]:
pk_hash = hash160(sender_pubkey)
scriptcode = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

input_amount_sat = int(2.001 * 100_000_000)
value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

hashPrevOuts = hash256(txid + index)
hashSequence = hash256(sequence)
hashOutputs = hash256(outputs)
sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

tx_digest_preimage = (
    version
    + hashPrevOuts
    + hashSequence
    + txid
    + index
    + varint_len(scriptcode)
    + scriptcode
    + value
    + sequence
    + hashOutputs
    + locktime
    + sighash_type
)
print(tx_digest_preimage.hex())

02000000ee7612c66f9ae792957ced87f2635ee395f6a7af6da67f71ae6572a6c0b6611b3bb13029ce7b1f559ef5e747fcac439f1455a2ec7c5f09b72290795e706650441c9973767718bacfff92dec7aeed87666880a1fb0a916c7ba912fa1e3150ced4000000001976a914fc7250a211deddc70ee5a2738de5f07817351cef88aca048ed0b00000000fffffffff91a6606c3037951dc241bb6edf31f9a61112940431aee9593c16c3bb3e1d2600000000001000000


Now we are ready to hash this transaction and produce an ecdsa signature on it. 

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [10]:
# Create sigHash to be signed
sighash = hash256(tx_digest_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(sender_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Witness field
witness = (
    # indicate the number of stack items for the txin
    # 2 items for signature and pubkey
    bytes.fromhex("02")
    + varint_len(signature)
    + signature
    + varint_len(sender_pubkey)
    + sender_pubkey
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  020000000001011c9973767718bacfff92dec7aeed87666880a1fb0a916c7ba912fa1e3150ced40000000000ffffffff0280d1f00800000000160014fc7250a211deddc70ee5a2738de5f07817351cef80f0fa0200000000160014531260aa2a199e228c537dfa42c82bea2c7c1f4d02483045022100c4be2b40dd0de4d8394c90dd48cf10f7b7ebb966a00ed174c2ee987994f6a2aa022018fb4447db30f5c572c189996c522b81ee337613a8012167718f91d11c8cdb0e0121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa00000000


### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [11]:
new_tx_txid = node.sendrawtransaction(signed_tx.hex())
# result = node.testmempoolaccept(rawtxs=[signed_tx.hex()])
print(new_tx_txid)

72ed59fd70134b2d5bb45eb16cb77d418ff2407998a30e72796aa775e437b05e


We can decode the serialized transaction using ```decoderawtransaction```. Notice that our output addresses match the change and receiver addresses from earlier.

In [12]:
print("receiver's p2wpkh address: " + receiver_address)
change_p2wpkh_addr = pk_to_p2wpkh(change_pubkey, network = "regtest")
print("sender's change p2wpkh address: " + change_p2wpkh_addr)

receiver's p2pkh address: bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw
sender's change p2pkh address: mo6CPsdW8EsnWdmSSCrQ6225VVDtpMBTug


In [13]:
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "72ed59fd70134b2d5bb45eb16cb77d418ff2407998a30e72796aa775e437b05e",
  "hash": "a5559bc4d2b421b0b6e0e96792d7548935ec2be929f66dc2af323408507125c5",
  "version": 2,
  "size": 223,
  "vsize": 141,
  "weight": 562,
  "locktime": 0,
  "vin": [
    {
      "txid": "d4ce50311efa12a97b6c910afba180686687edaec7de92ffcfba18777673991c",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "3045022100c4be2b40dd0de4d8394c90dd48cf10f7b7ebb966a00ed174c2ee987994f6a2aa022018fb4447db30f5c572c189996c522b81ee337613a8012167718f91d11c8cdb0e01",
        "034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa"
      ],
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "1.50000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "0 fc7250a211deddc70ee5a2738de5f07817351cef",
        "desc": "addr(bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw)#luvpmd0q",
        "hex": "0014fc7250a211deddc70ee5a2738de5

## Quiz

1. What fields to the transaction are introduced in segwit v0 transactions compared to legacy transactions?
2. When spending from a segwit v0 input, the sighash algorithm includes the input amount. For legacy outputs, the sighash does not include the amount. What advantage does this have for offline signers?
3. If you compare the transaction we created here to the transaction in the P2PKH notebook, you'll see it's roughly the same size in terms of the total number of bytes. So why are segwit transactions cheaper in terms of mining fees?

 ## Answers
    
1. The 'marker', 'flag', and 'witness' fields.
2. For legacy inputs, the offline signer would need to bet information from a bitcoin node to know how much the input was spending. By including the amount in the sighash, the signer can be sure that the input amount is what they specified. If the amount did not match, then the signature would be invalid. This makes it easier for the offline signer to know how much is being spent and how much the transaction is paying in miner fees.
3. Since the introduction of segwit, blocks are limited by 'weight units' rather than raw bytes. Each byte in the witness field counts for 1 weight unit, and bytes in the other fields count for 4 weight units. The relative discount for bytes in the witness field is what makes segwit inputs cheaper to spend from. See [BIP141](https://github.com/bitcoin/bips/blob/master/bip-0141.mediawiki).

## Exercise

Create a copy of `signed_tx` in a new variable (e.g. `signed_tx_copy`) and then use `node.decoderawtransaction(signed_tx_copy.hex())` to view the parsed transaction. Note the transaction ID. Try editing the signature in `signed_tx_copy` so that the signature is invalid but keep the length of the signature the same. Run `node.decoderawtransaction(signed_tx_copy.hex())` again to view the transaction ID of `signed_tx_copy`. Does the transaction ID change? Try editing another part of the transaction such as the output amount and view the new transaction ID again. 